<a href="https://colab.research.google.com/github/jathurT/Data-Crunch-Competition/blob/main/%20Notebooks_and_Scripts/Sample%20V1.0.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Data Analysis

In [ ]:
# Importing Libraries
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from prophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# Load data
train_df = pd.read_csv('/content/drive/MyDrive/DataCrunchCompetitionDatasets/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DataCrunchCompetitionDatasets/test.csv')
submission_df = pd.read_csv('/content/drive/MyDrive/DataCrunchCompetitionDatasets/sample_submission.csv')

In [13]:
# preview the dataset
train_df.head()

,ID,Year,Month,Day,kingdom,latitude,longitude,Avg_Temperature,Avg_Feels_Like_Temperature,Temperature_Range,Feels_Like_Temperature_Range,Radiation,Rain_Amount,Rain_Duration,Wind_Speed,Wind_Direction,Evapotranspiration
0,1,1,4,1,Arcadia,24.280002,-37.229980,25.50,30.50,8.5,10.3,22.52,58.89,16,8.6,283,1.648659
1,2,1,4,1,Atlantis,22.979999,-37.329990,299.65,305.15,5.9,8.2,22.73,11.83,12,15.8,161,1.583094
2,3,1,4,1,Avalon,22.880000,-37.130006,26.30,31.50,5.2,6.4,22.73,11.83,12,15.8,161,1.593309
3,4,1,4,1,Camelot,24.180003,-36.929994,24.00,28.40,8.2,10.7,22.67,75.27,16,6.4,346,1.638997
4,5,1,4,1,Dorne,25.780002,-37.530000,28.00,32.80,5.7,10.2,22.35,4.81,8,16.7,185,1.719189


In [14]:
# preview the dataset
test_df.head()

,ID,Year,Month,Day,kingdom
0,84961,9,1,1,Arcadia
1,84962,9,1,1,Atlantis
2,84963,9,1,1,Avalon
3,84964,9,1,1,Camelot
4,84965,9,1,1,Dorne


In [15]:
# preview the dataset
submission_df.head()

,ID,Avg_Temperature,Radiation,Rain_Amount,Wind_Speed,Wind_Direction
0,84961,0,0,0,0,0
1,84962,0,0,0,0,0
2,84963,0,0,0,0,0
3,84964,0,0,0,0,0
4,84965,0,0,0,0,0


In [16]:
train_df.shape

(84960, 17)

In [17]:
test_df.shape

(4530, 5)

In [18]:
submission_df.shape

(4530, 6)